# Research Data for paper "How accurate are developers' emotions inferred from social media?"

## Statistics - Multiple Linear Regression

This Notebook file contains the source codes for multiple linear regression over the dataset used in the paper "How accurate are developers' emotions inferred from social media?".

Due to participants' data privacy, we do not present any identifiable data here, such as participants' Twitter account or text publication.

In [42]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import random
import statistics
import seaborn as sns
import scipy.stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.model_selection import cross_val_score, cross_val_predict, RepeatedKFold 
from sklearn.linear_model import LinearRegression, LogisticRegression
import copy
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
import sklearn.metrics as me
from sklearn.metrics import classification_report
import pingouin as pg
from sklearn.preprocessing import StandardScaler, MinMaxScaler

## DATA READING

- The JSON file **'alldata.json'** contains the data used in this Notebook.

In [43]:
data = pd.read_json(r'https://raw.githubusercontent.com/leosilva/jss_2021_paper/master/alldata.json')

In [44]:
data.head()

,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44
participant_id,1,2,3,None,4,5,6,None,7,8,None,9,10,11,12,13,14,15,16
created_at,2009-03-11 12:51:34,2008-06-17 14:52:40,2017-11-05 01:21:27,2009-09-27 00:22:22,2015-08-18 09:44:42,2007-12-02 18:59:26,2018-08-28 15:53:30,2012-03-19 11:54:15,2008-01-09 20:56:00,2009-08-02 15:11:05,2008-12-21 15:24:53,2010-07-30 18:14:56,2009-01-04 16:46:32,2013-10-30 12:34:50,2018-08-07 07:54:08,2010-01-29 13:40:07,2018-09-15 09:25:06,2009-08-28 21:11:31,2010-03-04 19:05:54
tweets,"[[102566, 2021-03-31T19:13:28, 26, -0.45880000...","[[108111, 2021-03-31T20:30:21, 27, -0.0258, ne...","[[113189, 2021-03-31T21:52:37, 28, 0.0, neu, n...","[[119977, 2021-03-31T21:31:58, 29, 0.5859, pos...","[[123576, 2021-03-31T09:36:24, 30, 0.610300000...","[[126748, 2021-03-31T22:18:50, 31, 0.248100000...","[[139947, 2021-03-31T23:34:40, 32, 0.198400000...","[[142861, 2021-03-31T23:12:55, 33, -0.34, neg,...","[[154027, 2021-03-31T23:30:22, 34, 0.7088, pos...","[[157011, 2021-03-31T19:13:22, 35, -0.40190000...","[[160069, 2021-03-30T12:47:49, 36, 0.1027, pos...","[[163576, 2021-03-31T18:48:36, 37, 0.296, pos,...","[[171161, 2021-03-31T23:11:12, 38, 0.7096, pos...","[[176593, 2021-03-31T16:06:49, 39, -0.37160000...","[[179579, 2021-03-31T18:09:10, 40, 0.0, neu, n...","[[184846, 2021-03-31T17:53:40, 41, -0.2023, ne...","[[191406, 2021-03-31T07:12:32, 42, 0.0, neu, n...","[[196544, 2021-03-31T23:47:03, 43, 0.6369, pos...","[[224369, 2021-03-31T23:57:17, 44, -0.70890000..."
bigfive,"[[4, 37, 43, 36, 24, 26, 26]]","[[3, 41, 43, 34, 22, 23, 27]]","[[2, 43, 46, 31, 24, 15, 28]]",[],"[[9, 17, 33, 33, 22, 39, 30]]","[[1, 22, 33, 31, 20, 8, 31]]","[[14, 39, 49, 39, 28, 15, 32]]",[],"[[6, 35, 35, 32, 26, 16, 34]]","[[5, 29, 45, 34, 35, 29, 35]]",[],"[[7, 40, 46, 38, 22, 21, 37]]","[[8, 32, 46, 19, 18, 28, 38]]","[[10, 31, 36, 25, 33, 21, 39]]","[[11, 36, 45, 31, 31, 19, 40]]","[[12, 29, 41, 28, 22, 28, 41]]","[[13, 29, 43, 27, 23, 35, 42]]","[[15, 29, 45, 36, 27, 32, 43]]","[[16, 30, 35, 24, 20, 32, 44]]"


In [45]:
def create_tweets_df_per_user(data, id_user):

    tweets_pd = pd.DataFrame(data[id_user]['tweets'])
    tweets_pd.columns = ['id', "created_at", "id_user", "vader_sentiment_analysis_score", 
                         "vader_sentiment_analysis_polarity",
                         "vader_sentiment_analysis_polarity_5_cat",
                         "oplexicon_sentiment_analysis_score", "oplexicon_sentiment_analysis_polarity",
                         "oplexicon_sentiment_analysis_polarity_5_cat",
                         "sentistrength_sentiment_analysis_score", "sentistrength_sentiment_analysis_polarity",
                         "sentistrength_sentiment_analysis_polarity_5_cat",
                         "sentilexpt_sentiment_analysis_score", "sentilexpt_sentiment_analysis_polarity",
                         "sentilexpt_sentiment_analysis_polarity_5_cat",
                         "liwc_sentiment_analysis_score", "liwc_sentiment_analysis_polarity",
                         "liwc_sentiment_analysis_polarity_5_cat",
                         "final_score", "final_score_ensemble", 
                         "final_polarity", "final_polarity_ensemble", 
                         'text_updated', 'is_retweet', 'retweet_updated']
 
    return tweets_pd

In [46]:
all_tweets = pd.DataFrame()
for c in data:
    id_user = c
    if data[id_user]['bigfive']:
        tweets = create_tweets_df_per_user(data, id_user)
        columns_to_delete = ['id',
                             #"vader_sentiment_analysis_score", 
                             "vader_sentiment_analysis_polarity",
                             "vader_sentiment_analysis_polarity_5_cat",
                             #"oplexicon_sentiment_analysis_score",
                             "oplexicon_sentiment_analysis_polarity",
                             "oplexicon_sentiment_analysis_polarity_5_cat",
                             #"sentistrength_sentiment_analysis_score",
                             "sentistrength_sentiment_analysis_polarity",
                             "sentistrength_sentiment_analysis_polarity_5_cat",
                             #"sentilexpt_sentiment_analysis_score",
                             "sentilexpt_sentiment_analysis_polarity",
                             "sentilexpt_sentiment_analysis_polarity_5_cat",
                             #"liwc_sentiment_analysis_score",
                             "liwc_sentiment_analysis_polarity",
                             "liwc_sentiment_analysis_polarity_5_cat",
                             'text_updated', 'is_retweet', 'retweet_updated']

        for c in columns_to_delete:
            del tweets[c]

        tweets['Participant'] = data[id_user]['participant_id']
        if all_tweets.empty:
            all_tweets = tweets
        else:
            all_tweets = all_tweets.append(tweets)

In [47]:
all_tweets.head()

,created_at,id_user,vader_sentiment_analysis_score,oplexicon_sentiment_analysis_score,sentistrength_sentiment_analysis_score,sentilexpt_sentiment_analysis_score,liwc_sentiment_analysis_score,final_score,final_score_ensemble,final_polarity,final_polarity_ensemble,Participant
0,2021-03-31T19:13:28,26,-0.4588,0.000000,0.00,0.00,0.0000,-0.091760,0.000,neg,neu,1
1,2021-03-31T12:28:56,26,0.0000,0.000000,0.00,0.00,-0.0667,-0.013340,0.000,neg,neu,1
2,2021-03-31T12:17:06,26,0.7734,0.458831,0.00,0.00,0.0196,0.250366,0.000,pos,neu,1
3,2021-03-31T11:46:04,26,-0.8720,0.458831,0.25,-0.25,0.0400,-0.074634,0.000,neg,neu,1
4,2021-03-30T17:58:24,26,-0.6808,0.000000,0.00,-0.25,0.0000,-0.186160,-0.125,neg,neg,1


In [48]:
count = 1
json_data = {
    'Participant': [],
    'Id User': [],
    'O': [],
    'C': [],
    'E': [],
    'A': [],
    'N': []
}

for d in data:
    if data[d]['bigfive']:
        b = pd.DataFrame(data[d]['bigfive'])
        b.columns = ['id', 'O', 'C', 'E', 'A', 'N', 'Id User']
        b['Participant'] = count
        del b['id']
        b = b[['Participant', 'Id User', 'O', 'C', 'E', 'A', 'N']]
        json_data['Participant'].append(b['Participant'][0])
        json_data['Id User'].append(b['Id User'][0])
        json_data['O'].append(b['O'][0])
        json_data['C'].append(b['C'][0])
        json_data['E'].append(b['E'][0])
        json_data['A'].append(b['A'][0])
        json_data['N'].append(b['N'][0])
        count += 1
        
bf = pd.DataFrame(json_data)

In [49]:
bf.head()

,Participant,Id User,O,C,E,A,N
0,1,26,37,43,36,24,26
1,2,27,41,43,34,22,23
2,3,28,43,46,31,24,15
3,4,30,17,33,33,22,39
4,5,31,22,33,31,20,8


In [50]:
df_dict = {'id_user' : [],
            'pos' : [],
            'neu' : [],
            'neg' : [],
            'por_pos' : [],
            'por_neu' : [],
            'por_neg' : [],
            'mean' : [],
            'O': [],
            'C': [],
            'E': [],
            'A': [],
            'N': []
          }

df_grouped = all_tweets.groupby(['Participant'])
for i in range(1,len(df_grouped)+1):
    t = all_tweets.query('Participant == {}'.format(i))
    bigfive = bf.query('Participant == {}'.format(i))
    vader_df = t[['final_score_ensemble', 'final_polarity_ensemble']].copy()
    qtd_pos = vader_df.query('final_polarity_ensemble == "pos"').count()
    qtd_neu = vader_df.query('final_polarity_ensemble == "neu"').count()
    qtd_neg = vader_df.query('final_polarity_ensemble == "neg"').count()
    qtd_total = vader_df['final_polarity_ensemble'].count()
    df_dict['id_user'].append(bigfive['Id User'].iloc[0])
    df_dict['pos'].append(qtd_pos.iloc[0])
    df_dict['neu'].append(qtd_neu.iloc[0])
    df_dict['neg'].append(qtd_neg.iloc[0])
    df_dict['por_pos'].append(((qtd_pos / qtd_total) * 100).iloc[0])
    df_dict['por_neu'].append(((qtd_neu / qtd_total) * 100).iloc[0])
    df_dict['por_neg'].append(((qtd_neg / qtd_total) * 100).iloc[0])
    df_dict['mean'].append(vader_df['final_score_ensemble'].mean())

    df_dict['O'].append(bigfive['O'].iloc[0])
    df_dict['C'].append(bigfive['C'].iloc[0])
    df_dict['E'].append(bigfive['E'].iloc[0])
    df_dict['A'].append(bigfive['A'].iloc[0])
    df_dict['N'].append(bigfive['N'].iloc[0])

df = pd.DataFrame(df_dict)

biglist = ['O', 'C', 'E', 'A', 'N']
for b in biglist:
    res = []
    for r in df[b]:
        if r <= 10:
            res.append(1)
        elif r > 10 and r < 21:
            res.append(2)
        elif r > 20 and r < 31:
            res.append(3)
        elif r > 30 and r < 41:
            res.append(4)
        elif r > 40:
            res.append(5)

    df['{}_Coded'.format(b)] = res
df

,id_user,pos,neu,neg,por_pos,por_neu,por_neg,mean,O,C,E,A,N,O_Coded,C_Coded,E_Coded,A_Coded,N_Coded
0,26,1408,1866,1764,27.947598,37.038507,35.013894,-0.022431,37,43,36,24,26,4,5,4,3,3
1,27,1367,1926,1255,30.057168,42.348285,27.594547,-0.000698,41,43,34,22,23,5,5,4,3,3
2,28,1968,2031,1933,33.175995,34.238031,32.585974,-0.007192,43,46,31,24,15,5,5,4,3,2
3,30,1719,832,620,54.210028,26.237780,19.552192,0.113446,17,33,33,22,39,2,4,4,3,4
4,31,2993,1443,1557,49.941599,24.078091,25.980310,0.055160,22,33,31,20,8,3,4,4,2,1
5,32,4386,2693,2306,46.734150,28.694726,24.571124,0.057806,39,49,39,28,15,4,5,4,3,2
6,34,693,459,393,44.854369,29.708738,25.436893,0.051225,35,35,32,26,16,4,4,4,3,2
7,35,895,975,725,34.489403,37.572254,27.938343,0.028354,29,45,34,35,29,3,5,4,4,3
8,37,3235,1892,1329,50.108426,29.306072,20.585502,0.066105,40,46,38,22,21,4,5,4,3,3
9,38,1646,1336,1726,34.961767,28.377230,36.661003,-0.017339,32,46,19,18,28,4,5,2,2,3


## CORRELATIONS

In [51]:
df_corr = df[['por_pos', 'por_neu', 'por_neg', 'O', 'C', 'E', 'A', 'N', 'O_Coded', 'C_Coded', 'E_Coded', 'A_Coded', 'N_Coded']]
df_corr.rcorr(method='spearman', upper='pval', decimals=4, padjust=None, stars=False)

,por_pos,por_neu,por_neg,O,C,E,A,N,O_Coded,C_Coded,E_Coded,A_Coded,N_Coded
por_pos,-,0.0000,0.0061,0.6461,0.3571,0.6275,0.2172,0.6009,0.5271,0.0091,0.5931,0.1507,0.8498
por_neu,-0.8971,-,0.1373,0.4577,0.3250,0.8237,0.0613,0.5895,0.4378,0.0091,0.9206,0.0104,0.7564
por_neg,-0.6529,0.3882,-,0.9219,0.2489,0.655,0.7932,0.5895,0.6875,0.0210,0.4814,0.4594,0.8633
O,-0.1244,0.2,0.0267,-,0.0136,0.2654,0.6096,0.0316,0.0000,0.0988,0.3406,0.6508,0.1175
C,-0.2467,0.263,0.3061,0.6018,-,0.193,0.3163,0.4431,0.0468,0.0001,0.6738,0.5072,0.5072
E,0.1315,0.0606,-0.1211,0.2961,0.3433,-,0.1796,0.5119,0.6077,0.2006,0.0001,0.3391,0.6415
A,-0.3264,0.4778,-0.0712,0.1383,0.2676,0.3532,-,0.5336,0.5530,0.5118,0.1764,0.0000,0.4891
N,-0.1416,0.146,0.146,-0.5379,-0.2064,-0.177,-0.1682,-,0.0089,0.8714,0.1406,0.9025,0.0000
O_Coded,-0.1708,0.2088,0.1091,0.9368,0.5034,0.139,0.1604,-0.6297,-,0.2115,0.5051,0.5671,0.0389
C_Coded,-0.6289,0.6289,0.5704,0.4273,0.8128,0.3379,0.1771,0.044,0.3303,-,0.7406,0.4125,1.


In [52]:
columns_features = ['O', 'C', 'E', 'A', 'N']
df_features = pd.DataFrame(df[columns_features])
df_features

,O,C,E,A,N
0,37,43,36,24,26
1,41,43,34,22,23
2,43,46,31,24,15
3,17,33,33,22,39
4,22,33,31,20,8
5,39,49,39,28,15
6,35,35,32,26,16
7,29,45,34,35,29
8,40,46,38,22,21
9,32,46,19,18,28


In [53]:
target = pd.DataFrame(df[['por_pos', 'por_neu', 'por_neg']])
target

,por_pos,por_neu,por_neg
0,27.947598,37.038507,35.013894
1,30.057168,42.348285,27.594547
2,33.175995,34.238031,32.585974
3,54.210028,26.237780,19.552192
4,49.941599,24.078091,25.980310
5,46.734150,28.694726,24.571124
6,44.854369,29.708738,25.436893
7,34.489403,37.572254,27.938343
8,50.108426,29.306072,20.585502
9,34.961767,28.377230,36.661003


In [54]:
combs = []

def combinations(target,data):
    for i in range(len(data)):
        new_target = copy.copy(target)
        new_data = copy.copy(data)
        new_target.append(data[i])
        new_data = data[i+1:]
        combs.append(new_target)
        combinations(new_target, new_data)

In [70]:
df

,id_user,pos,neu,neg,por_pos,por_neu,por_neg,mean,O,C,E,A,N,O_Coded,C_Coded,E_Coded,A_Coded,N_Coded
0,26,1408,1866,1764,27.947598,37.038507,35.013894,-0.022431,37,43,36,24,26,4,5,4,3,3
1,27,1367,1926,1255,30.057168,42.348285,27.594547,-0.000698,41,43,34,22,23,5,5,4,3,3
2,28,1968,2031,1933,33.175995,34.238031,32.585974,-0.007192,43,46,31,24,15,5,5,4,3,2
3,30,1719,832,620,54.210028,26.237780,19.552192,0.113446,17,33,33,22,39,2,4,4,3,4
4,31,2993,1443,1557,49.941599,24.078091,25.980310,0.055160,22,33,31,20,8,3,4,4,2,1
5,32,4386,2693,2306,46.734150,28.694726,24.571124,0.057806,39,49,39,28,15,4,5,4,3,2
6,34,693,459,393,44.854369,29.708738,25.436893,0.051225,35,35,32,26,16,4,4,4,3,2
7,35,895,975,725,34.489403,37.572254,27.938343,0.028354,29,45,34,35,29,3,5,4,4,3
8,37,3235,1892,1329,50.108426,29.306072,20.585502,0.066105,40,46,38,22,21,4,5,4,3,3
9,38,1646,1336,1726,34.961767,28.377230,36.661003,-0.017339,32,46,19,18,28,4,5,2,2,3


In [71]:
scaler = StandardScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df))
df_scaled.columns = ['id_user','pos', 'neu', 'neg', 'por_pos', 'por_neu', 'por_neg',
                     'mean', 'O', 'C', 'E', 'A', 'N', 'O_Coded', 'C_Coded', 'E_Coded', 'A_Coded', 'N_Coded']
del df_scaled['id_user']
del df_scaled['mean']
del df_scaled['pos']
del df_scaled['neu']
del df_scaled['neg']
df_scaled

,por_pos,por_neu,por_neg,O,C,E,A,N,O_Coded,C_Coded,E_Coded,A_Coded,N_Coded
0,-0.946439,0.315281,1.448544,0.676419,0.29277,0.929890,-0.172629,0.222636,0.480384,0.67420,0.625543,0.000000,0.145865
1,-0.738330,1.035239,-0.030029,1.269444,0.29277,0.548397,-0.597562,-0.145865,1.761410,0.67420,0.625543,0.000000,0.145865
2,-0.430657,-0.064438,0.964693,1.565956,0.87831,-0.023843,-0.172629,-1.128534,1.761410,0.67420,0.625543,0.000000,-1.021055
3,1.644351,-1.149200,-1.632759,-2.288706,-1.65903,0.357650,-0.597562,1.819474,-2.081666,-1.48324,0.625543,0.000000,1.312785
4,1.223270,-1.442034,-0.351724,-1.547424,-1.65903,-0.023843,-1.022494,-1.988370,-0.800641,-1.48324,0.625543,-1.632993,-2.187975
5,0.906855,-0.816060,-0.632556,0.972931,1.46385,1.502130,0.677236,-1.128534,0.480384,0.67420,0.625543,0.000000,-1.021055
6,0.721415,-0.678569,-0.460020,0.379907,-1.26867,0.166903,0.252304,-1.005701,0.480384,-1.48324,0.625543,0.000000,-1.021055
7,-0.301090,0.387653,0.038485,-0.509631,0.68313,0.548397,2.164501,0.591137,-0.800641,0.67420,0.625543,1.632993,0.145865
8,1.239728,-0.733167,-1.426835,1.121188,0.87831,1.311384,-0.597562,-0.391532,0.480384,0.67420,0.625543,0.000000,0.145865
9,-0.254491,-0.859110,1.776790,-0.064862,0.87831,-2.312804,-1.447427,0.468303,0.480384,0.67420,-2.710687,-1.632993,0.145865


In [98]:
import statsmodels.tools.eval_measures as em
# ---------------------------------------------------------
# MULTIPLE LINEAR REGRESSION
# ---------------------------------------------------------
# EXECUTION: 10 times with 10-fold cross-validation
# TRAIN: 0.7
# TEST: 0.3
# ---------------------------------------------------------

def perform_multiple_linar_regression(dep_var, combs, df_scaled):
    res_json = {
        'dep_var': [],
        'ind_vars': [],
        'r2': [],
        'adj_r2': [],
        'rmse': [],
        'intercept':[],
        'coef':[],
        'coef_pvalues': []
    }

    for c in combs:
        X = df_scaled[c]
        y = df_scaled[dep_var]

        res_json['dep_var'].append(dep_var)
        res_json['ind_vars'].append(', '.join(c))

        scores = []
        scores_ols = []
        adj_r2 = []
        rmses = []
        coef_pvalues = []
        intercepts = []

        printed = False

        rkf = RepeatedKFold(n_splits=10, n_repeats=10, random_state=42)

        for train_index, test_index in rkf.split(X):
                X_train, X_test = X.loc[train_index], X.loc[test_index]
                y_train, y_test = y.loc[train_index], y.loc[test_index]

                regressor = LinearRegression()
                regressor.fit(X_train, y_train)

                y_pred = regressor.predict(X_test)

                X_train = sm.add_constant(X_train)
                m = sm.OLS(y_train, X_train).fit()
                predictions = m.predict(X_train)
                rmse = em.rmse(y_train, predictions)
                params = np.round(m.params,4)
                intercepts.append(params[0])
                scores_ols.append(m.rsquared)
                adj_r2.append(m.rsquared_adj)
                rmses.append(rmse)
                coef_pvalues.append(np.round(m.pvalues, 4))

        res_json['r2'].append(np.mean(scores_ols))
        res_json['adj_r2'].append(np.mean(adj_r2))
        res_json['rmse'].append(np.mean(rmses))
        res_json['intercept'].append(np.round(np.mean(intercepts), 4))
        res_json['coef'].append(np.round(params[1:].values, 4))
        coef_pvalues_dict = {}
        for p in coef_pvalues:
            for idx, pv in enumerate(p):
                if idx not in coef_pvalues_dict:
                    coef_pvalues_dict[idx] = []
                coef_pvalues_dict[idx].append(pv)

        coef_pvalues = []
        for cpd in coef_pvalues_dict:
            val = coef_pvalues_dict[cpd]
            coef_pvalues.append(np.round(np.mean(val), 4))

        res_json['coef_pvalues'].append(coef_pvalues)

    res_df = pd.DataFrame(res_json)
    res_df = res_df.sort_values(by=['adj_r2'], ascending=False)
    return res_df

In [99]:
res_df = perform_multiple_linar_regression('por_neg', combs, df_scaled)
res_df

/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda

/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda

/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda

/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda

/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda

...

/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda

/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda

/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda

/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda

,dep_var,ind_vars,r2,adj_r2,rmse,intercept,coef,coef_pvalues
21,por_neg,"C_Coded, E_Coded, A_Coded",0.517704,0.378385,0.688611,-0.0027,"[0.601, -0.3132, -0.2237]","[0.8039, 0.0223, 0.2879, 0.3441]"
9,por_neg,"C_Coded, E_Coded",0.467358,0.373786,0.724405,-0.0036,"[0.5502, -0.3914]","[0.8015, 0.0251, 0.1601]"
10,por_neg,"C_Coded, A_Coded",0.450437,0.353920,0.735626,-0.0020,"[0.6265, -0.3359]","[0.8002, 0.018, 0.1698]"
22,por_neg,"C_Coded, E_Coded, N_Coded",0.481081,0.331186,0.714652,-0.0068,"[0.5558, -0.4067, -0.0512]","[0.7977, 0.0308, 0.1646, 0.6767]"
29,por_neg,"C_Coded, E_Coded, A_Coded, N_Coded",0.526591,0.324836,0.682001,-0.0038,"[0.6037, -0.3233, -0.2208, -0.0304]","[0.804, 0.0301, 0.2955, 0.3934, 0.754]"
15,por_neg,"O_Coded, C_Coded, E_Coded",0.472773,0.320459,0.720619,-0.0043,"[-0.0556, 0.5667, -0.3898]","[0.802, 0.7958, 0.0504, 0.1738]"
25,por_neg,"O_Coded, C_Coded, E_Coded, A_Coded",0.522929,0.319580,0.684723,-0.0036,"[-0.038, 0.6117, -0.313, -0.2211]","[0.8063, 0.7907, 0.0475, 0.3045, 0.3719]"
23,por_neg,"C_Coded, A_Coded, N_Coded",0.454943,0.297525,0.732612,-0.0026,"[0.6186, -0.3342, 0.0693]","[0.7944, 0.0258, 0.1941, 0.8243]"
16,por_neg,"O_Coded, C_Coded, A_Coded",0.454380,0.296796,0.732907,-0.0033,"[-0.0395, 0.6376, -0.3332]","[0.7988, 0.8264, 0.0336, 0.1876]"
1,por_neg,C_Coded,0.336421,0.282839,0.809492,-0.0001,[0.5502],"[0.8053, 0.0327]"


In [100]:
res_df = perform_multiple_linar_regression('por_neu', combs, df_scaled)
res_df

/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda

/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda

/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda

/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda

/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda

...

/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda

/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda

/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda

/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda

,dep_var,ind_vars,r2,adj_r2,rmse,intercept,coef,coef_pvalues
10,por_neu,"C_Coded, A_Coded",0.555116,0.476996,0.661263,0.0006,"[0.4762, 0.4873]","[0.7945, 0.0538, 0.0425]"
21,por_neu,"C_Coded, E_Coded, A_Coded",0.588460,0.469639,0.636137,0.0103,"[0.4645, -0.1446, 0.5391]","[0.7631, 0.0809, 0.465, 0.0373]"
23,por_neu,"C_Coded, A_Coded, N_Coded",0.573292,0.450067,0.648632,-0.0030,"[0.4671, 0.4893, 0.0801]","[0.777, 0.0682, 0.0501, 0.6103]"
16,por_neu,"O_Coded, C_Coded, A_Coded",0.560478,0.433557,0.657205,0.0009,"[0.0181, 0.4712, 0.4861]","[0.7891, 0.8053, 0.0751, 0.0512]"
29,por_neu,"C_Coded, E_Coded, A_Coded, N_Coded",0.597090,0.425438,0.630080,0.0073,"[0.461, -0.1314, 0.5354, 0.0395]","[0.7562, 0.0967, 0.5606, 0.051, 0.7534]"
25,por_neu,"O_Coded, C_Coded, E_Coded, A_Coded",0.594021,0.421080,0.631686,0.0130,"[0.0187, 0.4592, -0.1446, 0.5379]","[0.7558, 0.7968, 0.1258, 0.49, 0.0469]"
27,por_neu,"O_Coded, C_Coded, A_Coded, N_Coded",0.579815,0.400792,0.643846,-0.0059,"[0.0723, 0.4436, 0.4851, 0.1079]","[0.7669, 0.7655, 0.1188, 0.0624, 0.6264]"
30,por_neu,"O_Coded, C_Coded, E_Coded, A_Coded, N_Coded",0.605636,0.370601,0.623351,0.0049,"[0.0491, 0.4454, -0.1246, 0.5301, 0.0605]","[0.7437, 0.7571, 0.163, 0.5774, 0.0654, 0.7397]"
3,por_neu,A_Coded,0.351736,0.299403,0.798066,0.0014,[0.5922],"[0.7865, 0.029]"
12,por_neu,"E_Coded, A_Coded",0.391032,0.284081,0.772179,0.0066,"[-0.1851, 0.6552]","[0.7786, 0.4838, 0.0285]"


In [103]:
res_df = perform_multiple_linar_regression('por_pos', combs, df_scaled)
res_df

/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda

/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda

/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda

/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda

/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda

...

/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda

/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda

/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda

/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/leosilva/opt/anaconda

,dep_var,ind_vars,r2,adj_r2,rmse,intercept,coef,coef_pvalues
21,por_pos,"C_Coded, E_Coded, A_Coded",0.585945,0.466403,0.637931,-0.0062,"[-0.6354, 0.2602, -0.2815]","[0.7851, 0.0197, 0.2626, 0.2432]"
1,por_pos,C_Coded,0.489663,0.448483,0.708394,-0.0005,[-0.6994],"[0.8046, 0.0062]"
10,por_pos,"C_Coded, A_Coded",0.524673,0.441231,0.683476,0.0006,"[-0.6566, -0.1883]","[0.8012, 0.0126, 0.3936]"
9,por_pos,"C_Coded, E_Coded",0.519232,0.434811,0.687318,-0.0049,"[-0.6994, 0.1618]","[0.7985, 0.0072, 0.4643]"
29,por_pos,"C_Coded, E_Coded, A_Coded, N_Coded",0.590945,0.416721,0.634087,-0.0034,"[-0.6343, 0.2556, -0.2802, -0.0137]","[0.7755, 0.0272, 0.3184, 0.2688, 0.7994]"
25,por_pos,"O_Coded, C_Coded, E_Coded, A_Coded",0.590273,0.415762,0.634569,-0.0077,"[0.0052, -0.6369, 0.2602, -0.2819]","[0.7781, 0.8114, 0.0445, 0.2861, 0.2643]"
11,por_pos,"C_Coded, N_Coded",0.500978,0.413367,0.700632,0.0004,"[-0.6899, -0.0877]","[0.7967, 0.0095, 0.677]"
23,por_pos,"C_Coded, A_Coded, N_Coded",0.537714,0.404264,0.674285,0.0035,"[-0.6461, -0.1906, -0.0926]","[0.7845, 0.0185, 0.4062, 0.6571]"
5,por_pos,"O_Coded, C_Coded",0.492992,0.403991,0.706114,0.0008,"[-0.011, -0.6961]","[0.8014, 0.8428, 0.0129]"
16,por_pos,"O_Coded, C_Coded, A_Coded",0.528646,0.392589,0.680692,0.0010,"[0.0064, -0.6584, -0.1887]","[0.7969, 0.8349, 0.0226, 0.4125]"
